### Env Config

In [1]:
%%capture
%pip install pandas requests fastparquet

In [2]:
import zipfile, requests, os
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

In [3]:
FLOAT_COLS = [
    "a_vencer_ate_90_dias",
    "a_vencer_de_91_ate_360_dias",
    "a_vencer_de_361_ate_1080_dias",
    "a_vencer_de_1081_ate_1800_dias",
    "a_vencer_de_1801_ate_5400_dias",
    "a_vencer_acima_de_5400_dias",
    "vencido_acima_de_15_dias",
    "carteira_ativa",
    "carteira_inadimplida_arrastada",
    "ativo_problematico"
]
CATEGORY_COLS = [
    'uf',
    'tcb',
    'sr',
    'cliente'
    'ocupacao',
    'cnae_secao',
    'cnae_subclasse',
    'cliente', 
    'ocupacao',
    'porte',
    'modalidade',
    'origem',
    'indexador'
]
INT_COLS = [
    'numero_de_operacoes'
]

def convert_category_dtype(df: pd.DataFrame, column: str):
    df[column] = df[column].astype('category')
    return df
def convert_float_dtype(df: pd.DataFrame, column: str):
    df[column] = df[column].apply(lambda x : str(x).replace(",","."))
    df[column] = df[column].astype('float64')
    return df
def convert_int_dtype(df: pd.DataFrame, column: str):
    df[column] = df[column].apply(lambda x : str(x).replace("<= 15","15"))
    df[column] = df[column].astype(int)
    return df

def export_to_parquet(df: pd.DataFrame, path: str):
	if not os.path.exists(path):
		os.makedirs(path)
	df.to_parquet(f'{path}/df.parquet', engine='fastparquet')

### Extração Manual e manutenção da integridade da zona Staging

In [7]:
planilhas = [
	'planilha_2012',
	'planilha_2013',
	'planilha_2014',
	'planilha_2015',
	'planilha_2016',
	'planilha_2017',
	'planilha_2018',
	'planilha_2019',
	'planilha_2020',
	'planilha_2021',
	'planilha_2022',
	'planilha_2023',
	'planilha_2024',
]

for planilha in planilhas:
	if not os.path.exists(f'data/staging/{planilha}.zip'):
		url = f'https://www.bcb.gov.br/pda/desig/{planilha}.zip'

		print(f'Baixando arquivo "{planilha}.zip"...')
		response = requests.get(url)

		with open(f'data/staging/{planilha}.zip', 'wb') as f:
			f.write(response.content)

Baixando arquivo "planilha_2012.zip"...


ConnectionError: HTTPSConnectionPool(host='www.bcb.gov.br', port=443): Max retries exceeded with url: /pda/desig/planilha_2012.zip (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002639335BF50>: Failed to resolve 'www.bcb.gov.br' ([Errno 11001] getaddrinfo failed)"))

### Exportando para zona Raw

In [5]:
if not os.path.exists('data/raw'):
	os.makedirs('data/raw')

for planilha in planilhas:
    zip_obj = zipfile.ZipFile(f'data/staging/{planilha}.zip', 'r')
    extracted_files = zip_obj.namelist()

    for file in extracted_files:
        extracted_path = os.path.join(f'data/raw', file)
        
        if not os.path.exists(extracted_path):
            print(f'Extraindo {file}...')
            try:
                zip_obj.extract(file, 'data/raw')
            except zipfile.BadZipFile:
                print(f"Erro: O arquivo {planilha}.zip está corrompido.")
            except Exception as e:
                print(f"Erro ao extrair {file}: {e}")

    zip_obj.close()

NameError: name 'planilhas' is not defined

### Criando dataframe

In [4]:
files = os.listdir('data/raw')
files.sort()
df = pd.DataFrame()
for file in files:
    temp_df = pd.read_csv(f'data/raw/{file}', sep=';', encoding='utf-8-sig', nrows=15000)
    df = pd.concat([df, temp_df])
    print(f'Processado o arquivo {file}')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa2 in position 7: invalid start byte

### Ajuste de tipagem dos dados

In [7]:
for column in df.columns:
	if column in FLOAT_COLS:
		df = convert_float_dtype(df=df, column=column)
		df = df.rename(columns={column: "vl_" + column.lower()})
	elif column in CATEGORY_COLS:
		df = convert_category_dtype(df=df, column=column)
		df = df.rename(columns={column: "ct_" + column.lower()})
	elif column in INT_COLS:
		df = convert_int_dtype(df=df, column=column)
		df = df.rename(columns={column: "nu_" + column.lower()})

In [8]:
df.dtypes

data_base                              object
ct_uf                                category
ct_tcb                               category
ct_sr                                category
ct_cliente                           category
ct_ocupacao                          category
ct_cnae_secao                        category
ct_cnae_subclasse                    category
ct_porte                             category
ct_modalidade                        category
ct_origem                            category
ct_indexador                         category
nu_numero_de_operacoes                  int64
vl_a_vencer_ate_90_dias               float64
vl_a_vencer_de_91_ate_360_dias        float64
vl_a_vencer_de_361_ate_1080_dias      float64
vl_a_vencer_de_1081_ate_1800_dias     float64
vl_a_vencer_de_1801_ate_5400_dias     float64
vl_a_vencer_acima_de_5400_dias        float64
vl_vencido_acima_de_15_dias           float64
vl_carteira_ativa                     float64
vl_carteira_inadimplida_arrastada 

In [9]:
df['dt_data_base'] = pd.to_datetime(df['data_base'].copy(), format="%Y-%m-%d")
df.drop(columns=['data_base'], inplace=True)

In [10]:
df['dt_data_base'].dtype.name

'datetime64[ns]'

In [11]:
df['ct_porte'].value_counts()

ct_porte
PJ - Micro                                       669342
PJ - Pequeno                                     563754
PJ - Médio                                       288752
PF - Mais de 5 a 10 salários mínimos              88965
PF - Mais de 3 a 5 salários mínimos               85810
PF - Mais de 1 a 2 salários mínimos               83242
PF - Mais de 2 a 3 salários mínimos               81845
PF - Mais de 10 a 20 salários mínimos             80033
PF - Acima de 20 salários mínimos                 75983
PF - Até 1 salário mínimo                         69558
PF - Sem rendimento                               38010
PJ - Grande                                       27588
PF - Indisponível                                 23280
PJ - Indisponível                                 13838
Name: count, dtype: int64

### Vamos separar coluna ct_porte em duas colunas:

ct_classificacao , que é o substring(0:2) de ct_porte

ct_tamanho , que é o substring(5:) de ct_porte

In [12]:
df['ct_classificacao'] = df['ct_porte'].apply(lambda x: x[0:2]).astype('category')
df['ct_porte'] = df['ct_porte'].apply(lambda x: x[5:].strip()).astype('category')

In [13]:
df['ct_modalidade'].value_counts()

ct_modalidade
PJ - Capital de giro                                                              391649
PJ - Outros créditos                                                              339850
PJ - Cheque especial e conta garantida                                            274245
PJ - Investimento                                                                 220932
PJ - Financiamento de infraestrutura/desenvolvimento/projeto e outros créditos    208129
PF - Outros créditos                                                              152418
PJ - Operações com recebíveis                                                     109568
PF - Cartão de crédito                                                            106873
PF - Empréstimo sem consignação em folha                                          101216
PF - Empréstimo com consignação em folha                                           78727
PF - Veículos                                                                      72736
PF - Ru

In [14]:
df['ct_modalidade'] = df['ct_modalidade'].apply(lambda x: x[5:].strip()).astype('category')

In [15]:
df['ct_ocupacao'].value_counts()

ct_ocupacao
-                                                  1563274
PF - Outros                                         116780
PF - Servidor ou empregado público                   98260
PF - Empresário                                      92430
PF - Autônomo                                        87336
PF - Empregado de empresa privada                    85104
PF - Aposentado/pensionista                          82621
PF - MEI                                             37027
PF - Empregado de entidades sem fins lucrativos      27168
Name: count, dtype: int64

In [16]:
df['ct_ocupacao'] = df['ct_ocupacao'].apply(lambda x: x[5:].strip()).astype('category')

In [17]:
df['vl_carteira_ativa'].describe()

count      2190000.00
mean       2439018.73
std       24629714.65
min              0.01
25%          12727.54
50%          66187.07
75%         330177.21
max     2088395883.83
Name: vl_carteira_ativa, dtype: float64

### Exportando para zona Trusted

In [18]:
export_to_parquet(df, 'data/trusted')

In [19]:
df['vl_carteira_ativa_n_arrastada'] = df['vl_carteira_ativa'] - df['vl_carteira_inadimplida_arrastada']

In [20]:
df['vl_carteira_ativa_n_arrastada'].describe()

count      2190000.00
mean       2346042.91
std       24145767.41
min              0.00
25%           9533.15
50%          56521.72
75%         300619.19
max     2088395883.83
Name: vl_carteira_ativa_n_arrastada, dtype: float64

In [21]:
df.dtypes

ct_uf                                      category
ct_tcb                                     category
ct_sr                                      category
ct_cliente                                 category
ct_ocupacao                                category
ct_cnae_secao                              category
ct_cnae_subclasse                          category
ct_porte                                   category
ct_modalidade                              category
ct_origem                                  category
ct_indexador                               category
nu_numero_de_operacoes                        int64
vl_a_vencer_ate_90_dias                     float64
vl_a_vencer_de_91_ate_360_dias              float64
vl_a_vencer_de_361_ate_1080_dias            float64
vl_a_vencer_de_1081_ate_1800_dias           float64
vl_a_vencer_de_1801_ate_5400_dias           float64
vl_a_vencer_acima_de_5400_dias              float64
vl_vencido_acima_de_15_dias                 float64
vl_carteira_

In [22]:
print((df['vl_a_vencer_acima_de_5400_dias'] != 0.00).sum())
print((df['vl_a_vencer_de_1801_ate_5400_dias'] != 0.00).sum())
print((df['vl_a_vencer_de_361_ate_1080_dias'] != 0.00).sum())
print((df['vl_a_vencer_de_91_ate_360_dias'] != 0.00).sum())
print((df['vl_a_vencer_de_1081_ate_1800_dias'] != 0.00).sum())
print((df['vl_a_vencer_ate_90_dias'] != 0.00).sum())

57704
340110
1338061
1804485
737369
1971449


In [23]:
CONVERSAO_FAIXAS = {
	'vl_a_vencer_acima_de_5400_dias': '> 180',
 	'vl_a_vencer_de_1801_ate_5400_dias': '36-180',
	'vl_a_vencer_de_1081_ate_1800_dias': '18-36',
	'vl_a_vencer_de_361_ate_1080_dias': '12-18',
	'vl_a_vencer_de_91_ate_360_dias': '3-12',
	'vl_a_vencer_ate_90_dias': '0-3',
}

def calcula_range(row):
    for col, val in CONVERSAO_FAIXAS.items():
        if row[col] != 0.00:
            return val
    return None

df['ct_faixa_meses_ate_vencimento'] = df.apply(calcula_range, axis=1).astype('category')

In [24]:
df['ct_faixa_meses_ate_vencimento'].value_counts()

ct_faixa_meses_ate_vencimento
12-18     603759
3-12      482856
18-36     401803
36-180    285139
0-3       253990
> 180      57704
Name: count, dtype: int64

In [25]:
df['vl_media_carteira_ativa_por_operacao'] = df['vl_carteira_ativa'] / df['nu_numero_de_operacoes']

In [26]:
df['vl_media_carteira_ativa_por_operacao'].describe()

count     2190000.00
mean        36578.75
std        826862.60
min             0.00
25%           708.67
50%          2937.60
75%         11163.86
max     139226392.26
Name: vl_media_carteira_ativa_por_operacao, dtype: float64

### Exportando para zona Refined

In [27]:
export_to_parquet(df, 'data/refined')